In [1]:
import pandas as pd

# Charger un fichier SAS (.sas7bdat)
df = pd.read_sas("../data/raw/credit.sas7bdat", format="sas7bdat", encoding="latin1")

# Afficher les premières lignes
print(df.head())

   Cle  Comptes  Duree_credit Historique_credit Objet_credit  Montant_credit  \
0  1.0      1.0           6.0               A34          A43          1169.0   
1  2.0      2.0          48.0               A32          A43          5951.0   
2  3.0      4.0          12.0               A34          A46          2096.0   
3  4.0      1.0          42.0               A32          A42          7882.0   
4  5.0      1.0          24.0               A33          A40          4870.0   

   Epargne  Anciennete_emploi  Taux_effort Situation_familiale  ...  \
0      0.0                5.0          4.0                 A93  ...   
1      1.0                3.0          2.0                 A92  ...   
2      1.0                4.0          2.0                 A93  ...   
3      1.0                4.0          2.0                 A93  ...   
4      1.0                3.0          3.0                 A93  ...   

  Anciennete_domicile  Biens   Age  Autres_credits Statut_domicile Nb_credits  \
0          